<a href="https://colab.research.google.com/github/Khairunnisah19/khairunnisah19.github.io/blob/main/Sentiment_Analysis_Review_Resep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Install library yg digunakan**

In [1]:
import numpy as np # Untuk Komputasi Numerik
import pandas as pd # Untuk Manipulasi Data
import matplotlib.pyplot as plt # Untuk Memvisualisasikan Data

#**2 . Menampilkan dataset**

In [2]:
df = pd.read_csv("sample_data/Recipe Reviews and User Feedback Dataset.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'sample_data/Recipe Reviews and User Feedback Dataset.csv'

#**3. Pre-Processing Data**

In [ ]:
import textblob
from textblob import TextBlob

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

review = df["text"].astype("string")

In [ ]:
def pre_process(Review_Text):
    # Case Folding: Lowercase
    Review_Text = str(Review_Text).lower()

    # Case Folding: Removing Number
    Review_Text = re.sub(r"\d+", "", Review_Text)

    # Removing Punctuation
    All_punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for elements in Review_Text:
      if elements in All_punct:
        Review_Text = Review_Text.replace(elements, "")

    #Case Folding: Removing Whitespace
    Review_Text = Review_Text.strip()

    #Separating Sentences with Split () Method
    pisah = Review_Text.split()

    #Tokenizing: Word Tokenizing Using NLTK Module
    tokens = nltk.tokenize.word_tokenize(Review_Text)

    removed = []
    for t in tokens:
      if t not in tokens:
        removed.append(t)

    return Review_Text

df['ReviewClean'] = df['text'].apply(lambda x:pre_process(x))

df.head()

#**4. Proses PeLabelan**

In [ ]:
# Perform a simple sentiment analysis
data_review = list(df['ReviewClean'].astype(str))
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, review in enumerate(data_review):
    analysis = TextBlob(review)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    elif analysis.sentiment.polarity == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')
df.head()

#**5. Menambahkan kolom Status Sentimen (Positif/Netral/Negatif)**

In [ ]:
df['klasifikasi'] = status
df.tail()

#**6. Data Visualization**

In [ ]:
def show_pie(label, data, legend_title) :
    fig, ax = plt.subplots(figsize=(8, 10), subplot_kw=dict(aspect='equal'))

    labels = [x.split()[-1] for x in label]

    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}% ({:d})".format(pct, absolute)

    wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                      textprops=dict(color="w"))

    ax.legend(wedges, labels,
              title= legend_title,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=10, weight="bold")
    plt.show()

label = ['Positif', 'Negatif', 'Netral']
count_data = [total_positif+1, total_negatif+1, total_netral]

show_pie(label, count_data, "Status")

**Kesimpulan :**

> Terdapat lebih banyak sentimen positif untuk resep masakan, dibandingkan dengan sentimen negatif dan netral



# **7. Klasifikasi Data Dengan Metode Logistic Regression dan SVM**

In [ ]:
#Vectorization
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(df['ReviewClean'])
X = cv.transform(df['ReviewClean'])
y = df['klasifikasi']

In [ ]:
#Build Classifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

In [ ]:
#Find the best value of C in logistic regression

for c in [0.01, 0.05, 0.25, 0.5, 1]:

    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print('Accuracy for C=%s: %s'
         % (c, accuracy_score(y_test, lr.predict(X_test))))

In [ ]:
#Find the best value of C in support vector

for c in [0.01, 0.05, 0.25, 0.5, 1]:

    sv = SVC(C=c)
    sv.fit(X_train, y_train)
    print('Accuracy for C=%s: %s'
         % (c, accuracy_score(y_test, sv.predict(X_test))))

In [ ]:
#Choose C=1 to build the final model for Logistic Regression.
final_model_lr = LogisticRegression(C=1)
final_model_lr.fit(X, y)
print('Akurasi Logistic Regression: %s' %accuracy_score(y_test, final_model_lr.predict(X_test)))

In [ ]:
#Choose C=1 to build the final model for Support Vector.
final_model_sv = SVC(C=1)
final_model_sv.fit(X, y)
print('Akurasi SVM: %s' %accuracy_score(y_test, final_model_sv.predict(X_test)))

#**8. Menyimpan tabel baru yang ada kolom klasifikasi/label sentimen**

In [ ]:
df.to_csv('labeled_text.csv', encoding='utf8', index=False)
